In [8]:
import pandas as pd
import plotly.express as px

# ---------------------------------------------------------------------
# BÖLÜM 1: VERİ HAZIRLIĞI VE YÜKLEME
# ---------------------------------------------------------------------

# Excel'den okumak istiyorsanız, bu kısmı kullanın:
# try:
#     # Dosya adını ve sayfa adını kontrol edin!
#     df_goals = pd.read_excel('super_lig_goller.xlsx')
# except FileNotFoundError:
#     print("Hata: Excel dosyası bulunamadı. Kod, içerideki dummy veriyi kullanacak.")
    
# Not: Excel dosyası olmadığı varsayımıyla, yukarıdaki büyük dummy veriyi
# doğrudan Python listesi olarak oluşturuyoruz.

data = [
    {'Team': 'Galatasaray', 'Player': 'Mauro Icardi', 'Goals': 3},
    {'Team': 'Galatasaray', 'Player': 'Hakim Ziyech', 'Goals': 1},
    {'Team': 'Galatasaray', 'Player': 'Barış Alper Y.', 'Goals': 2},
    {'Team': 'Fenerbahçe', 'Player': 'Edin Džeko', 'Goals': 4},
    {'Team': 'Fenerbahçe', 'Player': 'Dušan Tadić', 'Goals': 2},
    {'Team': 'Fenerbahçe', 'Player': 'Cengiz Ünder', 'Goals': 1},
    {'Team': 'Beşiktaş', 'Player': 'Vincent Aboubakar', 'Goals': 2},
    {'Team': 'Beşiktaş', 'Player': 'Rachid Ghezzal', 'Goals': 1},
    {'Team': 'Beşiktaş', 'Player': 'Ernest Muçi', 'Goals': 2},
    {'Team': 'Trabzonspor', 'Player': 'Paul Onuachu', 'Goals': 3},
    {'Team': 'Trabzonspor', 'Player': 'Trezeguet', 'Goals': 1},
    {'Team': 'Trabzonspor', 'Player': 'Anastasios B.', 'Goals': 2},
    {'Team': 'Başakşehir', 'Player': 'João Figueiredo', 'Goals': 2},
    {'Team': 'Başakşehir', 'Player': 'Jérôme Kemen', 'Goals': 1},
    {'Team': 'Başakşehir', 'Player': 'Dimitrios Pelkas', 'Goals': 2},
    {'Team': 'Adana Demirspor', 'Player': 'Younès Belhanda', 'Goals': 2},
    {'Team': 'Adana Demirspor', 'Player': 'Nani', 'Goals': 1},
    {'Team': 'Adana Demirspor', 'Player': 'Yusuf Erdoğan', 'Goals': 1},
    {'Team': 'Rizespor', 'Player': 'Alin Zeqiri', 'Goals': 2},
    {'Team': 'Rizespor', 'Player': 'Varesanović', 'Goals': 1},
    {'Team': 'Ankaragücü', 'Player': 'Ali Sowe', 'Goals': 3},
    {'Team': 'Ankaragücü', 'Player': 'Olimpiu Morutan', 'Goals': 1},
    {'Team': 'Kayserispor', 'Player': 'Mame Thiam', 'Goals': 2},
    {'Team': 'Kayserispor', 'Player': 'Miguel Cardoso', 'Goals': 1},
    {'Team': 'Kayserispor', 'Player': 'Guélor Kanga', 'Goals': 2},
    {'Team': 'Sivasspor', 'Player': 'Rey Manaj', 'Goals': 3},
    {'Team': 'Sivasspor', 'Player': 'Max Gradel', 'Goals': 1},
    {'Team': 'Sivasspor', 'Player': 'Mustapha Yatabaré', 'Goals': 1},
    {'Team': 'Kasımpaşa', 'Player': 'Mamadou Fall', 'Goals': 3},
    {'Team': 'Kasımpaşa', 'Player': 'Nuno da Costa', 'Goals': 1},
    {'Team': 'Pendikspor', 'Player': 'Oscar Romero', 'Goals': 2},
    {'Team': 'Pendikspor', 'Player': 'Mehmet Umut N.', 'Goals': 1},
]
df_goals = pd.DataFrame(data)


# ---------------------------------------------------------------------
# BÖLÜM 2: VERİ İŞLEME VE AGREGASYON
# ---------------------------------------------------------------------

# 2.1 Pie Grafiği İçin Takım Bazlı Toplam Golleri Hesaplama
# 'Team' sütununa göre grupla, 'Goals' sütunundaki değerleri topla.
df_total_goals = df_goals.groupby('Team')['Goals'].sum().reset_index()
df_total_goals.rename(columns={'Goals': 'Total_Goals'}, inplace=True)
# Sonuç: Her takım için tek bir toplam gol değeri içeren yeni DataFrame.


# 2.2 İnteraktif Hover Metni Oluşturma Fonksiyonu
def create_hover_text(team_name):
    """
    Belirtilen takım adına göre, o takımın tüm oyuncularını ve gollerini
    satır satır listeleyen bir metin oluşturur.
    """
    # Ana veri setinden sadece ilgili takımı filtrele
    team_data = df_goals[df_goals['Team'] == team_name]
    text = ""
    
    # Filtrelenmiş her satır (oyuncu) için bilgileri ekle
    for _, row in team_data.iterrows():
        # HTML satır atlama (<br>) yerine Python newline (\n) kullanıyoruz,
        # bu sayede hover şablonunda (hovertemplate) düzgün görünecek.
        text += f"{row['Player']}: {row['Goals']} gol\n" 
        
    return text.strip() # Baştaki/sondaki boşlukları kaldır


# 2.3 Oluşturulan Hover Metnini Ana DataFrame'e Ekleme
# Her takım adı için (df_total_goals içindeki), yukarıdaki fonksiyonu çalıştır
# ve sonucu 'Hover_Text' adında özel bir sütunda sakla.
df_total_goals['Hover_Text'] = df_total_goals['Team'].apply(create_hover_text)

df_total_goals['Hover_Text_HTML'] = df_total_goals['Hover_Text'].str.replace('\n', '<br>')


# ---------------------------------------------------------------------
# BÖLÜM 3: PLOTLY İLE GÖRSELLEŞTİRME
# ---------------------------------------------------------------------

# 3.1 Plotly Pie Grafiği Oluşturma
fig = px.pie(
    df_total_goals,
    values='Total_Goals', # Pasta dilimlerinin büyüklüğü (Toplam Gol)
    names='Team',        # Dilimlerin etiketleri (Takım Adı)
    title='Türkiye Süper Ligi - Takım Bazlı Gol Dağılımı (Örnek)',
    hole=0.3             # Donut grafiği yapmak için
)

# Custom Hover Template Ayarları (DÜZELTİLDİ)
fig.update_traces(
    # customdata olarak artık HTML etiketli metni kullanıyoruz
    customdata=df_total_goals[['Hover_Text_HTML']], 
    
    # hovertemplate içindeki <pre> etiketini kaldırdık.
    # customdata[0] içindeki <br> etiketleri alt alta listelemeyi sağlayacak.
    hovertemplate="<b>%{label}</b><br><br>" +             # Takım Adı (Kalın)
                  "Toplam Gol: %{value}<br>" +            # Takımın Toplam Golü
                  "Oyuncu Detayları:<br>" +
                  "%{customdata[0]}" +                    # HTML etiketli metni doğrudan ekler
                  "<extra></extra>"                       
)

# 3.3 Grafiği Görüntüleme (veya Dosyaya Kaydetme)
# Bu komut, kodu çalıştırdığınız ortamda (Jupyter, VS Code vb.) grafiği açar.
fig.show()

# Eğer bu grafiği Dash veya Streamlit'e aktaracaksanız fig objesini kullanırsınız.
# Eğer sadece JSON olarak kaydetmek isterseniz:
# fig.write_json("super_lig_goals_dashboard.json")

print("\n--- Grafik Oluşturuldu ---")
print("Kodunuz başarıyla çalıştı. Şimdi interaktif grafiği kontrol edebilirsiniz.")


--- Grafik Oluşturuldu ---
Kodunuz başarıyla çalıştı. Şimdi interaktif grafiği kontrol edebilirsiniz.
